# # ITLA Security Fest 
# # Pentesting Utilizando Inteligencia Artificial 

El objetivo de este taller es aplicar el uso de la inteligencia artificial (Deep Learning) para resolver captchas y poder automatizar la enumeración o explotación de servicios que tengan este control de seguridad. 

Esta presentación está basada en la investigación realizada por 
Adam Geitgey https://medium.com/@ageitgey/how-to-break-a-captcha-system-in-15-minutes-with-machine-learning-dbebb035a710

Este ejercicio nos permitirá resolver los captchas utilizando Deep Learning. 

# Introducción

El método que utilizaremos aplica para la resolver el captcha del componente "Really Simple CAPTCHA", un pluging de wordpress que es utilizado en más de un millon de paginas web. 

**Nota**: Si bien es cierto que el metodo puede ser utilizado para resolver otros captchas, solamente ha sido probado para resolver imagenes de este pluging y posiblemente necesite adecuaciones para resolver otros captchas.  

Para poder utilizar Deep Learning necesitaremos completar el siguiente proceso:

**1. Extraer cada letra/número de las imagenes del repositorio "Really Simple CAPTCHA"**, guardarlas para entrenar nuestro modelo.Cada imagen tiene 4 letras y/o números:

**2. Entrenar el modelo utilizando las imagenes individuales.**

**3. Utilizar el modelo para resolver captchas.**


# 1. - Extraer las Imagenes

Para entrenar nuestro modelo de deep learning, necesitamos enseñarle cómo se ve un número (1-9) y cómo se ven las letras del alfabeto (A-Z). Excluiremos el 0 y la i, ya que el programa de Really Simple Captcha no las utiliza, para evitar confusiones.

Haremos el ejercicio con una imagen para explicar el concepto y luego verificaremos todas las imagenes. 

![title](img/demo.png)

In [ ]:
# Importaremos los siguientes módulos:
import os
import os.path
import glob

# Estos modulos nos dan acceso a un grupo de funciones que nos permitirán trabajar con imagenes:
# Visualizarlas, leerlas, guardarlas, rotarlas, cambiar su tamaño y colores, etc.
import imutils
import cv2 
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

CAPTCHA_IMAGE = "generated_captcha_images/2A2X.png"
OUTPUT_FOLDER = "extracted_letter_images"


In [ ]:
# Dado que el nombre del archivo contiene el texto del captcha (es decir, "2A2X.png" tiene el texto "2A2X"),
# Tomemos el nombre del archivo base como el texto
filename = os.path.basename(CAPTCHA_IMAGE)
captcha_correct_text = os.path.splitext(filename)[0]

print(filename)
print(captcha_correct_text)

In [ ]:
# Cargue la imagen y conviértala a escala de grises
image = cv2.imread(CAPTCHA_IMAGE)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Genera una nueva figura
plt.figure()

# Mostrar la primera imagen en la columna izquierda
plt.subplot(1,2,1)
plt.imshow(image, cmap = plt.cm.binary)
plt.title('original')

# Desactivar numeración de ejes
plt.axis('off')

# Mostrar la segunda imagen en la columna derecha
plt.subplot(1,2,2)
plt.imshow(gray, cmap = plt.cm.binary)
plt.title('gray')

# Active la numeración del eje
plt.axis('on')

# Show the figure
plt.show()

In [ ]:
# Agregue un poco de relleno adicional alrededor de la imagen
gray = cv2.copyMakeBorder(gray, 8, 8, 8, 8, cv2.BORDER_REPLICATE)

plt.imshow(gray, cmap = plt.cm.binary)

In [ ]:
# Umbral de la imagen (convertirla a blanco y negro puro)
thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

plt.imshow(thresh, cmap = plt.cm.binary)

In [ ]:
# encontrar los contornos (manchas continuas de píxeles) de la imagen
contours = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Hack para compatibilidad con diferentes versiones de OpenCV
contours = contours[1] if imutils.is_cv3() else contours[0]
print("Numero de contornos = " + str(len(contours)))

# Ejemplo de Contorno:
img = gray.copy()
cv2.drawContours(img, contours, -1, (0,255,0), 3)
plt.imshow(img, cmap = plt.cm.binary)

In [ ]:
# Variable para guardar las letras 
letter_image_regions = []

# Ahora podemos recorrer cada uno de los cuatro contornos y extraer la letra
# dentro de cada uno
for contour in contours:
    # Obtener el rectángulo que contiene el contorno
    (x, y, w, h) = cv2.boundingRect(contour)

    # Compare el ancho y la altura del contorno para detectar letras que
    # están unidos en alguna parte
    if w / h > 1.25:
        # ¡Este contorno es demasiado ancho para ser una sola letra!
        # ¡Divídelo por la mitad en regiones de dos letras!
        half_width = int(w / 2)
        letter_image_regions.append((x, y, half_width, h))
        letter_image_regions.append((x + half_width, y, half_width, h))
    else:
        # Esta es una letra normal en sí misma
        letter_image_regions.append((x, y, w, h))

print(letter_image_regions)



In [ ]:
count = 0

# Guardar cada letra como una sola imagen
for letter_bounding_box, letter_text in zip(letter_image_regions, captcha_correct_text):
    # Toma las coordenadas de la letra en la imagen
    x, y, w, h = letter_bounding_box

    # Extraer la letra de la imagen original con un margen de 2 píxeles alrededor del borde
    letter_image = thresh[y - 2:y + h + 2, x - 2:x + w + 2]

    # Agrupar las imagenes para mostrarlas una por una. 
    plt.figure(count)
    plt.imshow(letter_image, cmap = plt.cm.binary)
    count = count + 1
    